# Building the CASBAH Galaxy Database (v1.0)

## SDSS

### Targeting
I am unclear on how to sensibly extract targeting information from the
SDSS.  But this may well be an issue for various analyses.

### Extracting Galaxy data
The script build_sdss loops through the listed fields with SDSS
coverage and calls the grab_sdss_spectra script to grab photometric
and spectral data.  It is currently grabbing DR12.

Here are some defaults:
* Box of 2deg on a side
* Photometry is Petrosian ugriz
* Galaxies are cut down to 20Mpc separation (LCDM cosmology)
* Stars are included (z=0)

Here is the basic procedure:
* Query photometry all objects in search box with spectra
* Query list of spectra from SDSS in search box
    * This list often contain duplicates from multiple passes
* Cut on 20Mpc using redshifts, RA+DEC of obj, and RA+DEC of QSO
* Loop on sources to build table
    * Take BOSS data over SDSS
* Generate a binary FITS table, including photometry, redshift and spectra

In [5]:
# Example call
from xastropy.casbah import galaxy_data as xcgd
reload(xcgd)
radec = (212.34957*u.deg,26.30585*u.deg)
tab=xcgd.grab_sdss_spectra(radec, radius=1.*u.degree/12.) 
Table(tab[0:5])

grab_sdss_spectra: Found 12 sources in the search box.
grab_sdss_spectra: Grabbing data for 12 sources.
Wrote SDSS table to None


RA,DEC,Z,Z_ERR,SDSS_MAG [5],SDSS_MAGERR [5],TELESCOPE,INSTRUMENT,FLUX [5000],SIG [5000],WAVE [5000]
float64,float64,float64,float64,float64,float64,string640,string640,float32,float32,float64
212.35102636,26.3415403908,3.21771526337,0.000997702358291,23.70004 .. 7.34031909445e+223,1.576056 .. 6.013470017e-154,SDSS 2.5-M,BOSS,3.5592 .. 1.35632e-19,2.65591 .. 1.35632e-19,3592.52587891 .. -1.22810024928e-296
212.349627859,26.3058613203,0.0,0.0,16.10909 .. -6.11925950261e-220,0.004779854 .. -2.78592920103e-305,SDSS 2.5-M,SDSS,0.0 .. 1.77053e-38,0.0 .. -1.21568e-37,3811.53564453 .. 1.57733159326e-52
212.370094746,26.3254555654,0.327256381512,5.58089996048e-05,21.36104 .. 6.013470017e-154,1.245788 .. 1.83037365249e-94,SDSS 2.5-M,BOSS,3.59836 .. -1.19132e+14,3.71348 .. -9.48227e+21,3590.87353516 .. 8.31176848954e+46
212.363198097,26.3766678271,0.673991203308,0.000221190101001,21.94763 .. 1.76807794921e+258,1.772229 .. -1.47312822905e-204,SDSS 2.5-M,BOSS,-0.0600585 .. 1.35632e-19,2.7654 .. -1.47499e+12,3592.52587891 .. 7.86200159392e-51
212.272722288,26.305233155,0.321305960417,8.39712593006e-05,21.33263 .. 3.00282156321e-142,0.844611 .. -1.19464315864e-195,SDSS 2.5-M,BOSS,-0.931015 .. 168.798,2.87149 .. 3.81592e+09,3590.87353516 .. -1.10999478584e-51


### Open questions/issues
1.  Should we be recording other observing meta-data?
1.  How about SFR, M*, etc.?

## DEIMOS

### Targeting

* Pull mask target info from Mask files :: parse_deimos_mask_file
* Pull other target info from SExtractor output
  * Requires yaml file describing target criteria
  * And the SExtractor output file

#### Sample output

MULTI_OBJ file:

|  INSTR |        MASK_NAME |     MASK_RA |   MASK_DEC | MASK_EPOCH | MASK_PA |    DATE_OBS | DISPERSER |   TEXP |         CONDITIONS |

| DEIMOS | PG1407_may_early | 14:09:34.10 | 26:18:45.1 |     2000.0 |   -96.1 | 23-Jul-2015 |      G600 | 3600.0 | POOR_SEEING,CLOUDS |

| DEIMOS | PG1407_may_early | 14:09:34.10 | 26:18:45.1 |     2000.0 |   -96.1 | 24-Jul-2015 |      G600 | 3600.0 |              CLEAR |


## Testing

In [48]:
ifile = '/Users/xavier/CASBAH/Galaxies/Observing/DEIMOS/PG1407+265_MULTI_OBJ.ascii'
mtab = Table.read(ifile,delimiter='|',format='ascii.fixed_width',
                 fill_values=[('--','0','DATE_OBS','TEXP')])
mtab

INSTR,MASK_NAME,MASK_RA,MASK_DEC,MASK_EPOCH,MASK_PA,DATE_OBS,DISPERSER,TEXP,CONDITIONS
string48,string128,string88,string80,float64,float64,string88,string32,float64,string144
DEIMOS,PG1407_may_early,14:09:34.10,26:18:45.1,2000.0,-96.1,23-Jul-2015,G600,3600.0,"POOR_SEEING,CLOUDS"
DEIMOS,PG1407_may_early,14:09:34.10,26:18:45.1,2000.0,-96.1,24-Jul-2015,G600,3600.0,CLEAR
DEIMOS,PG1407_may_mid,14:09:30.58,26:18:36.8,2000.0,80.0,--,--,--,--
DEIMOS,PG1407_may_mid2,14:09:30.58,26:16:36.8,2000.0,-2.0,--,--,--,--
DEIMOS,PG1407_may_late,14:09:30.58,26:16:36.8,2000.0,126.0,--,--,--,--


In [61]:
import yaml
yfile = '/Users/xavier/CASBAH/Galaxies/Observing/DEIMOS/PG1407+265/MASKS/PG1407+265_DEIMOS_targ.yaml'
stram = open(yfile, "r")
idict = yaml.load(stram)
idict

{'ALL_WITHIN_SEP': 20.0,
 'MAX_MAG': 25.0,
 'MAX_SEP': 10.0,
 'PHOTOM': 'MAG_BEST',
 'SEX_FILE': 'pg1407_sex_v.fits',
 'STAR_GAL': 0.9}

In [60]:
# Sextractor
sexfile = '/Users/xavier/CASBAH/Galaxies/Observing/DEIMOS/PG1407+265/MASKS/pg1407_sex_v.fits'
sex_tab = Table(fits.open(sexfile)[1].data)
sex_tab[0:5]


NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,FLUX_AUTO,FLUXERR_AUTO,FLUX_ISO,FLUXERR_ISO,MAG_BEST,MAGERR_BEST,MAG_ISO,MAGERR_ISO,MAG_AUTO,MAGERR_AUTO,FLUX_RADIUS,FWHM_IMAGE,FWHM_WORLD,FLAGS,ISOAREA_IMAGE,ELONGATION,CLASS_STAR,XWIN_IMAGE,YWIN_IMAGE,A_IMAGE,B_IMAGE,AWIN_IMAGE,BWIN_IMAGE,THETA_IMAGE,THETA_WORLD,KRON_RADIUS,MU_THRESHOLD
int32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32
1,7097.23,382.356,212.10933543,26.1178789051,4.78494e+07,5684.41,3.34956e+07,4566.22,13.3103,0.000129015,13.6975,0.000148047,13.3103,0.000129015,122.828,0.0,0.0,22,14807,5.69801,0.0286266,6758.95963395,217.39361991,79.8223,14.0088,88.3339,30.9015,88.5138,-88.4078,3.5,22.1499
2,7230.23,541.628,212.100060078,26.1278131743,1.83533e+07,4147.58,4.5603e+06,2361.47,14.3507,0.00024542,15.8625,0.000562366,14.3507,0.00024542,147.668,0.0,0.0,28,25720,8.83344,0.00050545,6751.00573249,230.944989433,123.324,13.961,102.415,35.3058,89.9967,-89.8864,3.5,25.7027
3,579.828,360.18,212.562790571,26.1165387359,-3.37058e+07,1679.65,5.9081e+06,2163.88,99.0,99.0,15.5814,0.000397755,99.0,99.0,-1519.26,0.0,0.0,4,11296,3.08167,0.796922,579.827760258,360.180052818,44.2252,14.351,44.2252,14.351,89.0801,-89.1734,0.0,25.0785
4,5952.85,131.343,212.188978836,26.1023083797,2.89552e+09,45437.5,3.68823e+07,5926.46,8.85568,1.70419e-05,13.593,0.000174505,8.85568,1.70419e-05,1548.7,0.0,0.0,20,121573,39.4626,0.0853614,3619.39698224,-660.373053443,555.554,14.078,555.554,14.078,1.91545e-05,0.0716425,26.0599,25.7027
5,4863.93,131.318,212.264731507,26.1023714279,-4.43195e+06,1061.98,3.38949e+06,1594.25,99.0,99.0,16.1847,0.000510801,99.0,99.0,-275.852,0.0,0.0,4,5234,1.27767,0.813222,4863.93002233,131.318298664,18.0119,14.0974,18.0119,14.0974,0.0930789,-0.0545487,0.0,25.3141


In [62]:
from astropy.table import MaskedColumn

In [67]:
tmp = MaskedColumn(['None','blah'],mask=[True,False],name='tst')

In [68]:
tmp

--
blah
